**PyImageJ Tutorial**

This notebook covers how to use ImageJ as a library from Python. A major advantage of this approach is the ability to combine ImageJ with other tools available from the Python software ecosystem, including NumPy, SciPy, scikit-image, CellProfiler, OpenCV, ITK and more.

This notebook assumes familiarity with the ImageJ API. Detailed tutorials in that regard can be found in the other notebooks.

# 1 Starting ImageJ from Python

The [pyimagej](https://pypi.org/project/pyimagej/) module enables access to the entire ImageJ API from Python in a natural way.

Let's initialize an ImageJ gateway including Fiji plugins, at a reproducible version:

In [1]:
import imagej

# initialize ImageJ
ij = imagej.init('sc.fiji:fiji:2.5.0')
print(f"ImageJ version: {ij.getVersion()}")

12:13:52.713 [SciJava-22635d99-Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
	at loci.formats.ClassList.<init>(ClassList.java:122)
	at loci.formats.ImageReader.getDefaultReaderClasses(ImageReader.java:80)
	at io.scif.bf.BioFormatsFormat.cacheReaderClasses(BioFormatsFormat.java:538)
	at io.scif.bf.BioFormatsFormat.<init>(BioFormatsFormat.java:139)
	at sun.reflect.NativeConstructorAccessorImpl.newInsta

## 1.1 Ways to initialize

| Requirement                                            | Code<sup>1</sup>                                                  | Reproducible?<sup>2</sup> |
|:-------------------------------------------------------|:------------------------------------------------------------------|:-------------------------:|
| Newest available version of ImageJ2 w/ ImageJ support  | `ij = imagej.init()`                                              | NO                        |
| Specific version of ImageJ2 w/ ImageJ support          | `ij = imagej.init('2.5.0')`                                       | YES                       |
| Interactive (newest version)*                          | `ij = imagej.init(mode='interactive')`                            | NO                        |
| Interactive (specific version)*                        | `ij = imagej.init('net.imagej:imagej:2.5.0', mode='interactive')` | YES                       |
| Without support for original ImageJ (newest versions)  | `ij = imagej.init('net.imagej:imagej', add_legacy=False`          | NO                        |
| With Fiji plugins (newest version)                     | `ij = imagej.init('sc.fiji:fiji')`                                | NO                        |
| With Fiji plugins (specific version)                   | `ij = imagej.init('sc.fiji:fiji:2.5.0')`                          | YES                       |
| From a local installation                              | `ij = imagej.init('/Applications/Fiji.app')`                      | DEPENDS                   |

*: _`mode='interactive'` is unavailalbe on macOS. Instead use `mode='gui'`. When set to `gui` mode the Python interpeter is blocked and no longer interactive. Check out the [initialization](Initialization.md) documentation for more information._

<sup>1</sup> pyimagej uses [`jgo`](https://github.com/scijava/jgo) internally to call up ImageJ, so all of these initializations are tied to the usage of `jgo`. You can read up on the [usage of `jgo`](https://github.com/scijava/jgo#usage) to find out more about this initialization.

<sup>2</sup> ___Reproducible___ means code is stable, executing the same today, tomorrow, and in years to come. While it is convenient and elegant to depend on the newest version of a program, behavior may change when new versions are released—for the better if bugs are fixed; for the worse if bugs are introduced—and people executing your notebook at a later time may encounter broken cells, unexpected results, or other more subtle behavioral differences. You can help avoid this pitfall by pinning to a specific version of the software. The British Ecological Society published [Guide to Better Science: Reproducible Code](https://www.britishecologicalsociety.org/wp-content/uploads/2018/12/BES-Reproducible-Code.pdf) diving into the relevant challenges in more detail, including an [R](https://www.r-project.org/)-centric illustration of best practices. A web search for `reproducible python` also yields several detailed articles.

## 1.2 The ImageJ2 gateway

The ImageJ2 gateway is the object interface that lets you use ImageJ-related
features (see [Initialization.md](Initialization.md)).  This gateway contains
all of the regular ImageJ2 Java functions. PyImageJ also adds a module of
convenience functions under `ij.py`. For example, converting a numpy array to
an ImageJ2 dataset:

In [2]:
import numpy as np

array = np.random.rand(5, 4, 3)
dataset = ij.py.to_java(array)

print(dataset.shape)

(3, 4, 5)


## 1.3 Increasing the memory available to Java

Java's virtual machine (the JVM) has a "max heap" value limiting how much
memory it can use. You can increase it:

```python
import imagej
import scyjava
scyjava.config.add_options('-Xmx6g')
ij = imagej.init()
```

Replace `6g` with the amount of memory Java should have. You can also pass
[other JVM arguments](https://docs.oracle.com/javase/8/docs/technotes/tools/unix/java.html).

Without having specified the max heap value explicitly, here is how much memory this notebook's JVM has available:

In [3]:
ij.getApp().getInfo(True)

'ImageJ2 2.5.0/1.53r; Java 1.8.0_312 [amd64]; 25MB of 7072MB'